In [1]:
import socket
import socketserver, time
from threading import Thread
from time import sleep
import sys

In [2]:
host = '25.93.32.9'  #http://25.94.218.230:555
port = 1234

timeout = 180
buffer_size = 1024

CONFIRMATION = bytes('Data processed', 'utf-8')
NO_DATA = bytes('no_data', 'utf-8')

# broadcast = ('<broadcast>', 1234)

messages = []

In [3]:
def control(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)

In [4]:
def setup():
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
    
    server.settimeout(timeout)
    server.bind((host, port))

    print('Server connected...')
    
    return server

In [5]:
def receive_msg(server, fail_state_count = 0):
    
    while True:
        bytes_received, address = server.recvfrom(1024)

        message = bytes_received.decode()

        print('message: ', message)
        print('from: ', address)

        if message[0] == 'S':
            fail_state_count = 0

            byte_value = bytes(message[1:], 'utf-8')

            bytes_to_send = int.from_bytes(byte_value, 'little')

            messages.append(bytes_to_send)

            server.sendto(CONFIRMATION, address)

        elif message[0] == 'A':

            if len(messages) != 0:

                to_send = messages[0]
                messages.pop(0)

                MV = control(to_send, 0, 10, 100, 0)

                bytes_to_send = int.to_bytes(MV, 4, byteorder = 'little')
                server.sendto(bytes_to_send, address)

            else:
                server.sendto(NO_DATA, address)
                fail_state_count = fail_state_count + 1 
                
                print('No data sent to actuator')

                if fail_state_count == 3:
                    server.sendto(int.to_bytes(0, 4, 'little'), address)
                    print('Fail state activated')
                    sys.exit(1)

        else:
            print('Cant interpretate request')
            
        time.sleep(0.5)

In [6]:
server = setup()

receive_msg(server)

Server connected...
message:  A
from:  ('25.93.32.9', 61546)
No data sent to actuator
message:  A
from:  ('25.93.32.9', 61546)
No data sent to actuator
message:  A
from:  ('25.93.32.9', 61546)
No data sent to actuator


SystemExit: 1

C:\Users\eduar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# int.to_bytes(5,4,byteorder='little')
# a = int.to_bytes(5, 4, byteorder = 'little')
# s = b's' + a
# int.from_bytes(a, 'little')